In [1]:
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense



2025-05-12 22:59:59.041545: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747090799.222301      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747090799.275805      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
max_lines = 50000
lines = []
file_path = '/kaggle/input/preprocessed-arwiki-for-autocomplete-txt/preprocessed_arwiki_for_autocomplete.txt'

with open(file_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= max_lines:
            break
        lines.append(line.strip())

text = ' '.join(lines)


In [3]:
tokens = re.findall(r'\b\w+\b', text)
print("number of :", len(tokens))

number of : 928357


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)

sequences = []
for line in lines:
    tokenized_line = tokenizer.texts_to_sequences([line])[0]
    for i in range(2, len(tokenized_line) + 1):  # n-grams from 2 tokens
        sequences.append(tokenized_line[:i])

# Pad sequences
max_len = max(len(seq) for seq in sequences)
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

# Split to X (inputs) and y (labels)
X = sequences[:, :-1]
y = sequences[:, -1]


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LambdaCallback
import tensorflow as tf

# تعريف الموديل
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50))  # تقليل الأبعاد
model.add(LSTM(64))  # أخف من 128
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# إعداد الـ optimizer
optimizer = Adam(learning_rate=0.001)  # lr ثابت
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer)

# تعريف الـ callback لحساب Perplexity بعد كل epoch
def calculate_perplexity(epoch, logs):
    if 'loss' in logs:
        perplexity = np.exp(logs['loss'])
        print(f"Epoch {epoch + 1}: Perplexity = {perplexity:.4f}")

perplexity_callback = LambdaCallback(on_epoch_end=calculate_perplexity)



I0000 00:00:1747090823.005445      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1747090823.006164      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [6]:

model.fit(X, y, epochs=3, batch_size=128, verbose=1, callbacks=[perplexity_callback])


Epoch 1/3


I0000 00:00:1747090829.486283      98 cuda_dnn.cc:529] Loaded cuDNN version 90300


6988/6988 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 9.8787Epoch 1: Perplexity = 15363.7149
6988/6988 ━━━━━━━━━━━━━━━━━━━━ 284s 40ms/step - loss: 9.8787
Epoch 2/3
6987/6988 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 8.8970Epoch 2: Perplexity = 6677.5227
6988/6988 ━━━━━━━━━━━━━━━━━━━━ 283s 40ms/step - loss: 8.8970
Epoch 3/3
6987/6988 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 8.1313Epoch 3: Perplexity = 3337.2858
6988/6988 ━━━━━━━━━━━━━━━━━━━━ 283s 40ms/step - loss: 8.1313


In [7]:
def predict_next_word(context_words):
    if isinstance(context_words, str):
        context_words = context_words.strip().split()

    seq = tokenizer.texts_to_sequences([' '.join(context_words)])
    padded = pad_sequences(seq, maxlen=X.shape[1]) 
    pred = model.predict(padded, verbose=0)
    next_index = np.argmax(pred)
    
    for word, index in tokenizer.word_index.items():
        if index == next_index:
            return word
    return ""


In [8]:
context = ["اللغة", "العربية"]
print("الكلمة المتوقعة:", predict_next_word(context))

الكلمة المتوقعة: محمد
